In [8]:
#***************** SCRIPT GLOBAL ***********************#
# Importar librerias
import pandas as pd
from gurobipy import * # Libreria de Solver Gurobi 
import numpy as np # Libreria para Matrices
import warnings

# Lista de Provinicias.
P = ["AZUAY", "LOJA", "IMBABURA", "BOLIVAR", "CAÑAR", "CARCHI", "CHIMBORAZO", "COTOPAXI", "EL ORO", "ESMERALDAS", "GALAPAGOS", "GUAYAS", "LOS RIOS", "MANABI", "MORONA SANTIAGO", "NAPO", "ORELLANA", "PASTAZA", "PICHINCHA", "SANTA ELENA", "SANTO DOMINGO DE LOS TSACHILAS","SUCUMBIOS", "TUNGURAHUA", "ZAMORA CHINCHIPE"]
## Tasa de Poblacion
T1 = 8/100000 
T2 = 316 

## Modelo para cada provincia lee cada excel 
for p in P:
    ## Poblacion 
    Cantones = list(pd.read_excel(p +".xlsx", sheet_name='Frecuencia')["Nom_Can"])
    Poblacion = dict()
    i = 0 # Contador cantones
    for c in Cantones: 
        Poblacion[c] = list(pd.read_excel( p +".xlsx", sheet_name='Frecuencia')["Poblacion"])[i]
        i=i+1
    ## Demanda
    df = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})
    Fiscalias = list(df["Name"]) # Nombre de Fiscalías
    Grupos = ["Grupo1","Grupo2", "Grupo3" ]
    Demanda = tupledict({})
    Nombres_Y = tuplelist([])
    Nombres_X = tuplelist([])
    for c in Cantones:
        df1 = pd.read_excel( p +".xlsx", sheet_name=c)
        for g in Grupos: 
            j = 0 # Contador Fiscalías
            for f in Fiscalias:
                Nombres_Y.append((c,Fiscalias[j]))
                j=j+1
                
    Nombres_Y =list(set(Nombres_Y))
    for c in Cantones:
        j = 0
        df1 = pd.read_excel( p +".xlsx", sheet_name=c)
        sum1=0
        sum2=0
        sum3=0
        for f in Fiscalias:
            if list(df1["Grupo1"])[j]+list(df1["Grupo2"])[j]+list(df1["Grupo3"])[j]<= 158:
                for g in Grupos:
                    Demanda[c,g,f] = 0
                sum1 = sum1 + list(df1["Grupo1"])[j]
                sum2 = sum2 + list(df1["Grupo2"])[j]
                sum3 = sum3 + list(df1["Grupo3"])[j]
            else:
                for g in Grupos: 
                    Demanda[c,g,f] = list(df1[g])[j]
            j=j+1
        Demanda[c,"Grupo1","MULTICOMPETENTE"] = sum1  
        Demanda[c,"Grupo2","MULTICOMPETENTE"] = sum2  
        Demanda[c,"Grupo3","MULTICOMPETENTE"] = sum3  

    Nombres_X = list(Demanda.keys())
    ## Carga Laboral
    Carga = tupledict({})
    df1 = pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')

    i = 0      # Contador de Cantones
    for c in Cantones:
        for f in Fiscalias:
            if f == 'FISCALIA DE SOLUCIONES RAPIDAS':
                Carga[c,f] = round(list(df1["TASA_SR"])[i])
                i=i+1
            else:
                Carga[c,f] = 316 
    print(Carga)
    ## Multicompetente 
    Multi = dict() 
    i = 0    # Contador de Cantones
    for c in Cantones:
        Multi[c] = list(df1["MULTICOMPETENTE"])[i]
        i=i+1
        
    de =[] # Demandas pronosticadas total 
    for c in Cantones:
        t = pd.read_excel( p +".xlsx", sheet_name=c)
        de.append(t["Grupo1"].sum()+ t["Grupo2"].sum() + t["Grupo3"].sum())

    ## *********************** CREACION DEL MODELO ***********************************

    ## Creamos el CONSTRUCTOR DEL MODELO 
    m = Model('Modelo_FGE')

    ## Creamos las Variables
    X = m.addVars(Nombres_X , name="", vtype=GRB.INTEGER) 
    Y = m.addVars(Nombres_Y , name="Fiscales_Necesarios", vtype=GRB.INTEGER)

    # Creacion de la FUNCIÓN OBJETIVO
    m.setObjective((Y.sum(Cantones,Fiscalias)) , GRB.MINIMIZE)

    ## Creacion de las Restricciones 
    # Restricción 1
    for c in Cantones:
        if Multi[c] == 0:
            for f in Fiscalias:
                m.addConstr((X.sum(c,Grupos,f) <= Y[c,f] * Carga[c,f]) , "tasa de carga")
        elif Multi[c] == 1:
            m.addConstr((X.sum(c, Grupos,Fiscalias) <= Y.sum(c,Fiscalias)*T2 ) , "tasa de carga_{}".format(f))

    # Restriccion 2      
    for f in Fiscalias:
        for c in Cantones:
            for g in Grupos:
                m.addConstr((X[c,g,f] == Demanda[c,g,f]), "demandadelitos")
                
    # Restriccion 3
   # for c in Cantones:
    #    m.addConstr(Y.sum(c,Fiscalias) >= Poblacion[c]* T1, "Se cumpla tasa T1")

    m.optimize()
    warnings.simplefilter("ignore")
    ## Tabla de Resultados 1 
    Tabla1 = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})
    for c in Cantones:
        Res = []
        for f in Fiscalias:
            Res.append(Y[c,f].x) 
        Tabla1[c] = Res

    for c in Cantones:
        if Multi[c] == 0:
            Tabla1[c][1] = Tabla1[c][1]+Tabla1[c][8]
            Tabla1[c][8] = 0
        elif Multi[c] == 1:
            suma = 0 
            for j in range(len(list(Tabla1[c]))):
                suma = suma + Tabla1[c][j]
                Tabla1[c][j] = 0
            Tabla1[c][8] = suma

    ## Tabla de Resultados 2 
    Tabla2 = pd.DataFrame({'Cantones': Cantones})
    Fo = []
    for c in Cantones:
        Fo.append(Tabla1[c].sum())

    Tabla2["Poblacion"] = list(pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')["Poblacion"])
    Tabla2["Delitos"] = de
    Tabla2["Fiscales Actuales"] = list(pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')["Fisc_Actuales"])
    Tabla2["Fiscales x Carga"] = Fo
    Tabla2["Déficit x Carga"] = Tabla2["Fiscales x Carga"]-Tabla2["Fiscales Actuales"] 
    Tabla2["Tasa Actual"] = (Tabla2["Fiscales Actuales"] * 100000)/Tabla2["Poblacion"]
    Tabla2["Tasa x Carga"] = (Tabla2["Fiscales x Carga"] * 100000)/Tabla2["Poblacion"]

    
######################################## MODEL 2

    m = Model('Modelo_FGE')

    ## Creamos las Variables
    X = m.addVars(Nombres_X , name="", vtype=GRB.INTEGER) 
    Y = m.addVars(Nombres_Y , name="Fiscales_Necesarios", vtype=GRB.INTEGER)

    # Creacion de la FUNCIÓN OBJETIVO
    m.setObjective((Y.sum(Cantones,Fiscalias)) , GRB.MINIMIZE)

    ## Creacion de las Restricciones 
# Restricción 1
    for c in Cantones:
        if Multi[c] == 0:
            for f in Fiscalias:
                m.addConstr((X.sum(c,Grupos,f) <= Y[c,f] * Carga[c,f]) , "tasa de carga")
        elif Multi[c] == 1:
            m.addConstr((X.sum(c, Grupos,Fiscalias) <= Y.sum(c,Fiscalias)*T2 ) , "tasa de carga_{}".format(f))

    # Restriccion 2      
    for f in Fiscalias:
        for c in Cantones:
            for g in Grupos:
                m.addConstr((X[c,g,f] == Demanda[c,g,f]), "demandadelitos")

    # Restriccion 3
    for c in Cantones:
        m.addConstr(Y.sum(c,Fiscalias) >= Poblacion[c]* T1, "Se cumpla tasa T1")
    
    m.optimize()
    Tabla1 = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})

    warnings.simplefilter("ignore")
    ## Tabla de Resultados 1 
    for c in Cantones:
        Res = []
        for f in Fiscalias:
            Res.append(Y[c,f].x) 
        Tabla1[c] = Res

    for c in Cantones:
        if Multi[c] == 1:
            suma = 0 
            for j in range(len(list(Tabla1[c]))):
                suma = suma + Tabla1[c][j]
                Tabla1[c][j] = 0
            Tabla1[c][8] = suma
    ## Tabla de Resultados 2 
    Fo = []
    for c in Cantones:
        Fo.append(Tabla1[c].sum())

    Tabla2["Fiscales x Población"] = Fo
    Tabla2["Déficit x Población"] = Tabla2["Fiscales x Población"]-Tabla2["Fiscales Actuales"] 
    Tabla2["Tasa x Población"] = (Tabla2["Fiscales x Población"] * 100000)/Tabla2["Poblacion"]
    print(Tabla2)
    
    dfNew = list(Tabla2["Déficit x Carga"])
    dfNew1 = list(Tabla2["Déficit x Población"] - Tabla2["Déficit x Carga"])
    k=0
    for i in dfNew1:
        if i < 0:
            dfNew1[k] = 0  
        k=k+1    
        
    Tabla1.loc[9] = ["Déficit x Carga"] + dfNew    
    Tabla1.loc[10] = ["Faltantes x Población"] + dfNew1 
    
    writer = pd.ExcelWriter('Resultados_' + p + '.xlsx')
    Tabla1.to_excel(writer, sheet_name='Fiscalias', index=False)
    Tabla2.to_excel(writer, sheet_name='Resumen', index=False)
    writer.save()
    writer.close()

{('CUENCA', 'PERSONAS Y GARANTIAS'): 316, ('CUENCA', 'DELINCUENCIA ORGANIZADA'): 316, ('CUENCA', 'ADMINISTRACION PUBLICA'): 316, ('CUENCA', 'SOLUCIONES RAPIDAS'): 316, ('CUENCA', 'VIOLENCIA DE GENERO'): 316, ('CUENCA', 'PATRIMONIO CIUDADANO'): 316, ('CUENCA', 'FE  PUBLICA'): 316, ('CUENCA', 'ACCIDENTES DE TRANSITO'): 316, ('CUENCA', 'MULTICOMPETENTE'): 316, ('GUALACEO', 'PERSONAS Y GARANTIAS'): 316, ('GUALACEO', 'DELINCUENCIA ORGANIZADA'): 316, ('GUALACEO', 'ADMINISTRACION PUBLICA'): 316, ('GUALACEO', 'SOLUCIONES RAPIDAS'): 316, ('GUALACEO', 'VIOLENCIA DE GENERO'): 316, ('GUALACEO', 'PATRIMONIO CIUDADANO'): 316, ('GUALACEO', 'FE  PUBLICA'): 316, ('GUALACEO', 'ACCIDENTES DE TRANSITO'): 316, ('GUALACEO', 'MULTICOMPETENTE'): 316, ('CAMILO PONCE ENRIQUEZ', 'PERSONAS Y GARANTIAS'): 316, ('CAMILO PONCE ENRIQUEZ', 'DELINCUENCIA ORGANIZADA'): 316, ('CAMILO PONCE ENRIQUEZ', 'ADMINISTRACION PUBLICA'): 316, ('CAMILO PONCE ENRIQUEZ', 'SOLUCIONES RAPIDAS'): 316, ('CAMILO PONCE ENRIQUEZ', 'VIOLENCIA

{('LOJA', 'PERSONAS Y GARANTIAS'): 316, ('LOJA', 'DELINCUENCIA ORGANIZADA'): 316, ('LOJA', 'ADMINISTRACION PUBLICA'): 316, ('LOJA', 'SOLUCIONES RAPIDAS'): 316, ('LOJA', 'VIOLENCIA DE GENERO'): 316, ('LOJA', 'PATRIMONIO CIUDADANO'): 316, ('LOJA', 'FE  PUBLICA'): 316, ('LOJA', 'ACCIDENTES DE TRANSITO'): 316, ('LOJA', 'MULTICOMPETENTE'): 316, ('CATAMAYO', 'PERSONAS Y GARANTIAS'): 316, ('CATAMAYO', 'DELINCUENCIA ORGANIZADA'): 316, ('CATAMAYO', 'ADMINISTRACION PUBLICA'): 316, ('CATAMAYO', 'SOLUCIONES RAPIDAS'): 316, ('CATAMAYO', 'VIOLENCIA DE GENERO'): 316, ('CATAMAYO', 'PATRIMONIO CIUDADANO'): 316, ('CATAMAYO', 'FE  PUBLICA'): 316, ('CATAMAYO', 'ACCIDENTES DE TRANSITO'): 316, ('CATAMAYO', 'MULTICOMPETENTE'): 316, ('CALVAS', 'PERSONAS Y GARANTIAS'): 316, ('CALVAS', 'DELINCUENCIA ORGANIZADA'): 316, ('CALVAS', 'ADMINISTRACION PUBLICA'): 316, ('CALVAS', 'SOLUCIONES RAPIDAS'): 316, ('CALVAS', 'VIOLENCIA DE GENERO'): 316, ('CALVAS', 'PATRIMONIO CIUDADANO'): 316, ('CALVAS', 'FE  PUBLICA'): 316, (

{('IBARRA', 'PERSONAS Y GARANTIAS'): 316, ('IBARRA', 'DELINCUENCIA ORGANIZADA'): 316, ('IBARRA', 'ADMINISTRACION PUBLICA'): 316, ('IBARRA', 'SOLUCIONES RAPIDAS'): 316, ('IBARRA', 'VIOLENCIA DE GENERO'): 316, ('IBARRA', 'PATRIMONIO CIUDADANO'): 316, ('IBARRA', 'FE  PUBLICA'): 316, ('IBARRA', 'ACCIDENTES DE TRANSITO'): 316, ('IBARRA', 'MULTICOMPETENTE'): 316, ('OTAVALO', 'PERSONAS Y GARANTIAS'): 316, ('OTAVALO', 'DELINCUENCIA ORGANIZADA'): 316, ('OTAVALO', 'ADMINISTRACION PUBLICA'): 316, ('OTAVALO', 'SOLUCIONES RAPIDAS'): 316, ('OTAVALO', 'VIOLENCIA DE GENERO'): 316, ('OTAVALO', 'PATRIMONIO CIUDADANO'): 316, ('OTAVALO', 'FE  PUBLICA'): 316, ('OTAVALO', 'ACCIDENTES DE TRANSITO'): 316, ('OTAVALO', 'MULTICOMPETENTE'): 316, ('ANTONIO ANTE', 'PERSONAS Y GARANTIAS'): 316, ('ANTONIO ANTE', 'DELINCUENCIA ORGANIZADA'): 316, ('ANTONIO ANTE', 'ADMINISTRACION PUBLICA'): 316, ('ANTONIO ANTE', 'SOLUCIONES RAPIDAS'): 316, ('ANTONIO ANTE', 'VIOLENCIA DE GENERO'): 316, ('ANTONIO ANTE', 'PATRIMONIO CIUDAD

Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 204 rows, 252 columns and 441 nonzeros
Model fingerprint: 0x827778fd
Variable types: 0 continuous, 252 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+02]
Found heuristic solution: objective 20.0000000
Presolve removed 204 rows and 252 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 211 rows, 252 columns and 504 nonzeros
Model fingerprint: 0xe108f556

{('TULCAN', 'PERSONAS Y GARANTIAS'): 316, ('TULCAN', 'DELINCUENCIA ORGANIZADA'): 316, ('TULCAN', 'ADMINISTRACION PUBLICA'): 316, ('TULCAN', 'SOLUCIONES RAPIDAS'): 316, ('TULCAN', 'VIOLENCIA DE GENERO'): 316, ('TULCAN', 'PATRIMONIO CIUDADANO'): 316, ('TULCAN', 'FE  PUBLICA'): 316, ('TULCAN', 'ACCIDENTES DE TRANSITO'): 316, ('TULCAN', 'MULTICOMPETENTE'): 316, ('MONTUFAR', 'PERSONAS Y GARANTIAS'): 316, ('MONTUFAR', 'DELINCUENCIA ORGANIZADA'): 316, ('MONTUFAR', 'ADMINISTRACION PUBLICA'): 316, ('MONTUFAR', 'SOLUCIONES RAPIDAS'): 316, ('MONTUFAR', 'VIOLENCIA DE GENERO'): 316, ('MONTUFAR', 'PATRIMONIO CIUDADANO'): 316, ('MONTUFAR', 'FE  PUBLICA'): 316, ('MONTUFAR', 'ACCIDENTES DE TRANSITO'): 316, ('MONTUFAR', 'MULTICOMPETENTE'): 316, ('BOLIVAR', 'PERSONAS Y GARANTIAS'): 316, ('BOLIVAR', 'DELINCUENCIA ORGANIZADA'): 316, ('BOLIVAR', 'ADMINISTRACION PUBLICA'): 316, ('BOLIVAR', 'SOLUCIONES RAPIDAS'): 316, ('BOLIVAR', 'VIOLENCIA DE GENERO'): 316, ('BOLIVAR', 'PATRIMONIO CIUDADANO'): 316, ('BOLIVAR

...

{('QUITO', 'PERSONAS Y GARANTIAS'): 316, ('QUITO', 'DELINCUENCIA ORGANIZADA'): 316, ('QUITO', 'ADMINISTRACION PUBLICA'): 316, ('QUITO', 'SOLUCIONES RAPIDAS'): 316, ('QUITO', 'VIOLENCIA DE GENERO'): 316, ('QUITO', 'PATRIMONIO CIUDADANO'): 316, ('QUITO', 'FE  PUBLICA'): 316, ('QUITO', 'ACCIDENTES DE TRANSITO'): 316, ('QUITO', 'MULTICOMPETENTE'): 316, ('RUMINAHUI', 'PERSONAS Y GARANTIAS'): 316, ('RUMINAHUI', 'DELINCUENCIA ORGANIZADA'): 316, ('RUMINAHUI', 'ADMINISTRACION PUBLICA'): 316, ('RUMINAHUI', 'SOLUCIONES RAPIDAS'): 316, ('RUMINAHUI', 'VIOLENCIA DE GENERO'): 316, ('RUMINAHUI', 'PATRIMONIO CIUDADANO'): 316, ('RUMINAHUI', 'FE  PUBLICA'): 316, ('RUMINAHUI', 'ACCIDENTES DE TRANSITO'): 316, ('RUMINAHUI', 'MULTICOMPETENTE'): 316, ('MEJIA', 'PERSONAS Y GARANTIAS'): 316, ('MEJIA', 'DELINCUENCIA ORGANIZADA'): 316, ('MEJIA', 'ADMINISTRACION PUBLICA'): 316, ('MEJIA', 'SOLUCIONES RAPIDAS'): 316, ('MEJIA', 'VIOLENCIA DE GENERO'): 316, ('MEJIA', 'PATRIMONIO CIUDADANO'): 316, ('MEJIA', 'FE  PUBLIC

Variable types: 0 continuous, 108 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 6e+02]
Found heuristic solution: objective 31.0000000
Presolve removed 100 rows and 108 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 31 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.100000000000e+01, best bound 3.100000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 103 rows, 108 columns and 216 nonzeros
Model fingerprint: 0x47ffca7e
Variable types: 0 continuous, 108 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0

Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 204 rows, 252 columns and 441 nonzeros
Model fingerprint: 0x375334a5
Variable types: 0 continuous, 252 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 8e+02]
Found heuristic solution: objective 24.0000000
Presolve removed 204 rows and 252 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 24 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.400000000000e+01, best bound 2.400000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 211 rows, 252 columns and 504 nonzeros
Model fingerprint: 0xb463ab90

Found heuristic solution: objective 53.0000000
Presolve removed 269 rows and 324 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 53 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.300000000000e+01, best bound 5.300000000000e+01, gap 0.0000%
                       Cantones  Poblacion  Delitos  Fiscales Actuales  \
0                        AMBATO     395893     8537                 24   
1          SAN PEDRO DE PELILEO      68404      822                  1   
2           BANOS DE AGUA SANTA      25940      596                  0   
3                        PATATE      16171      294                  1   
4                         QUERO      20696      256                  1   
5                       TISALEO      14647      199                  0   
6                      CEVALLOS      10233      181                  0   
7        

8                   1.0                  1.0         13.702384  


In [9]:
Y['ZAMORA','MULTICOMPETENTE'].x


2.0